In [1]:
import keras
import tensorflow as tf
import mlxtend
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import skimage
from mlxtend.evaluate import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, Dense, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint   
from sklearn.metrics import classification_report  
from skimage.feature import greycomatrix
from keras.models import load_model

In [2]:
seed = 7
np.random.seed(seed)
np.set_printoptions(precision = 3, suppress=True)

In [3]:
img_width, img_height = 256, 256
batch_size = 40
epochs = 100                                 
train_samples = 1200
validation_samples = 400
test_samples = 400

In [4]:
train_data_dir = './data/train/'
validation_data_dir = './data/validate/'
test_data_dir = './data/test/'

In [5]:
def co_occurrence_horiz(image):
    r_horiz = skimage.feature.graycomatrix(np.uint64(image[:,:,0]), [1], [0], levels=256, normed=True)
    g_horiz = skimage.feature.graycomatrix(np.uint64(image[:,:,1]), [1], [0], levels=256, normed=True)
    b_horiz = skimage.feature.graycomatrix(np.uint64(image[:,:,2]), [1], [0], levels=256, normed=True)
    co_occurrence_horiz_img = np.dstack((r_horiz[:,:,0], g_horiz[:,:,0], b_horiz[:,:,0]))
    return co_occurrence_horiz_img 

In [6]:
datagen = ImageDataGenerator(preprocessing_function = co_occurrence_horiz)

In [7]:
train_generator = datagen.flow_from_directory(
        train_data_dir,
        color_mode="rgb",
        target_size=(img_width, img_height),
        batch_size=batch_size,
        shuffle=False,
        class_mode='categorical')
validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        color_mode="rgb",
        target_size=(img_width, img_height),
        batch_size=batch_size,
        shuffle=False,
        class_mode='categorical')
test_generator = datagen.flow_from_directory(
        test_data_dir,
        color_mode="rgb",
        target_size=(img_width, img_height),
        batch_size= 1,
        shuffle=False,
        class_mode='categorical')
train_generator.class_indices

Found 1199 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


{'gan': 0, 'real': 1}

In [8]:
train_samples = train_generator.samples
validation_samples = validation_generator.samples
test_samples = test_generator.samples

In [9]:
model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape=(img_width, img_height, 3), activation='relu'))
model.add(Convolution2D(32, (5, 5)))
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(Convolution2D(64, (5, 5)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(Convolution2D(128, (5, 5)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten()) 
model.add(Dense(256))
model.add(Dense(256))
model.add(Dense(2, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.000001), 
              metrics=['accuracy'])

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-11-22 20:42:56.361169: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-11-22 20:42:56.361577: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 250, 250, 32)      25632     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 125, 125, 32)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 123, 123, 64)      18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 119, 119, 64)      102464    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 59, 59, 64)       0         
 2D)                                                    

In [10]:
checkpointer = ModelCheckpoint(filepath='./model/fake_image_detector.h5',
                               verbose=1,
                               save_best_only=True)

In [11]:
history = model.fit(train_generator,
                              steps_per_epoch=train_samples / batch_size,
                              epochs=epochs, callbacks=[checkpointer],
                              validation_data=validation_generator,
                              validation_steps=validation_samples / batch_size)

Epoch 1/100


2022-11-22 20:42:57.744868: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-22 20:42:58.078527: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


30/29 [==============================] - ETA: 0s - loss: 0.6932 - accuracy: 0.4579

2022-11-22 20:43:43.779389: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_loss improved from inf to 0.69312, saving model to ./model/fake_image_detector.h5
29/29 [==============================] - 54s 2s/step - loss: 0.6932 - accuracy: 0.4579 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/100
30/29 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5004
Epoch 2: val_loss improved from 0.69312 to 0.69310, saving model to ./model/fake_image_detector.h5
29/29 [==============================] - 51s 2s/step - loss: 0.6931 - accuracy: 0.5004 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/100
30/29 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5004
Epoch 3: val_loss improved from 0.69310 to 0.69307, saving model to ./model/fake_image_detector.h5
29/29 [==============================] - 52s 2s/step - loss: 0.6931 - accuracy: 0.5004 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/100
30/29 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5046
Epoch 4: val_loss improved from 0.

Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Internal Error (0000000e:Internal Error)
	<AGXG13GFamilyCommandBuffer: 0x29e1c8c00>
    label = <none> 
    device = <AGXG13GDevice: 0x14b7c5800>
        name = Apple M1 
    commandQueue = <AGXG13GFamilyCommandQueue: 0x11b416c00>
        label = <none> 
        device = <AGXG13GDevice: 0x14b7c5800>
            name = Apple M1 
    retainedReferences = 1


30/29 [==============================] - ETA: 0s - loss: 0.6926 - accuracy: 0.5038
Epoch 11: val_loss improved from 0.69261 to 0.69247, saving model to ./model/fake_image_detector.h5
29/29 [==============================] - 59s 2s/step - loss: 0.6926 - accuracy: 0.5038 - val_loss: 0.6925 - val_accuracy: 0.5000
Epoch 12/100
30/29 [==============================] - ETA: 0s - loss: 0.6925 - accuracy: 0.6347
Epoch 12: val_loss improved from 0.69247 to 0.69232, saving model to ./model/fake_image_detector.h5
29/29 [==============================] - 59s 2s/step - loss: 0.6925 - accuracy: 0.6347 - val_loss: 0.6923 - val_accuracy: 0.5350
Epoch 13/100
30/29 [==============================] - ETA: 0s - loss: 0.6923 - accuracy: 0.5013
Epoch 13: val_loss improved from 0.69232 to 0.69211, saving model to ./model/fake_image_detector.h5
29/29 [==============================] - 58s 2s/step - loss: 0.6923 - accuracy: 0.5013 - val_loss: 0.6921 - val_accuracy: 0.5125
Epoch 14/100
 8/29 [=======>..........

Error: command buffer exited with error status.
	The Metal Performance Shaders operations encoded on it may not have completed.
	Error: 
	(null)
	Internal Error (0000000e:Internal Error)
	<AGXG13GFamilyCommandBuffer: 0x29e1b35c0>
    label = <none> 
    device = <AGXG13GDevice: 0x14b7c5800>
        name = Apple M1 
    commandQueue = <AGXG13GFamilyCommandQueue: 0x11b416c00>
        label = <none> 
        device = <AGXG13GDevice: 0x14b7c5800>
            name = Apple M1 
    retainedReferences = 1


30/29 [==============================] - ETA: 0s - loss: 0.6921 - accuracy: 0.5021
Epoch 14: val_loss improved from 0.69211 to 0.69183, saving model to ./model/fake_image_detector.h5
29/29 [==============================] - 59s 2s/step - loss: 0.6921 - accuracy: 0.5021 - val_loss: 0.6918 - val_accuracy: 0.5000
Epoch 15/100
30/29 [==============================] - ETA: 0s - loss: 0.6918 - accuracy: 0.5096
Epoch 15: val_loss improved from 0.69183 to 0.69157, saving model to ./model/fake_image_detector.h5
29/29 [==============================] - 60s 2s/step - loss: 0.6918 - accuracy: 0.5096 - val_loss: 0.6916 - val_accuracy: 0.5450
Epoch 16/100
30/29 [==============================] - ETA: 0s - loss: 0.6915 - accuracy: 0.6372
Epoch 16: val_loss improved from 0.69157 to 0.69122, saving model to ./model/fake_image_detector.h5
29/29 [==============================] - 61s 2s/step - loss: 0.6915 - accuracy: 0.6372 - val_loss: 0.6912 - val_accuracy: 0.5125
Epoch 17/100
30/29 [==================